### Setup inicial

In [1]:
# setup de bibliotecas utilizadas
import random
import math
import json
import pandas as pd
random.seed(123)

### Lista de dicionários com os dados do enunciado

In [2]:
# definição das zonas com rentabilidade esperada e impacto ambiental conforme apresentado no enunciado
zonas = [
    {"zona": "A1", "rentabilidade": 2600, "impacto": 2},
    {"zona": "A2", "rentabilidade": 3400, "impacto": 4},
    {"zona": "A3", "rentabilidade": 1900, "impacto": 3},
    {"zona": "A4", "rentabilidade": 2100, "impacto": 4},
    {"zona": "A5", "rentabilidade": 3000, "impacto": 2},
    {"zona": "A6", "rentabilidade": 3200, "impacto": 2},
    {"zona": "A7", "rentabilidade": 3400, "impacto": 1},
    {"zona": "A8", "rentabilidade": 4300, "impacto": 2},
    {"zona": "A9", "rentabilidade": 4700, "impacto": 4},
    {"zona": "A10", "rentabilidade": 1100, "impacto": 3},
    {"zona": "A11", "rentabilidade": 4600, "impacto": 4},
    {"zona": "A12", "rentabilidade": 4700, "impacto": 4},
    {"zona": "A13", "rentabilidade": 1700, "impacto": 1},
    {"zona": "A14", "rentabilidade": 5000, "impacto": 4},
    {"zona": "A15", "rentabilidade": 1900, "impacto": 1},
    {"zona": "A16", "rentabilidade": 2900, "impacto": 1},
    {"zona": "A17", "rentabilidade": 1300, "impacto": 4},
    {"zona": "A18", "rentabilidade": 3300, "impacto": 1},
    {"zona": "A19", "rentabilidade": 3000, "impacto": 1},
    {"zona": "A20", "rentabilidade": 4200, "impacto": 1},
    {"zona": "A21", "rentabilidade": 4200, "impacto": 4},
    {"zona": "A22", "rentabilidade": 2400, "impacto": 1},
    {"zona": "A23", "rentabilidade": 4600, "impacto": 3},
    {"zona": "A24", "rentabilidade": 1800, "impacto": 1},
    {"zona": "A25", "rentabilidade": 4500, "impacto": 4},
    {"zona": "A26", "rentabilidade": 2000, "impacto": 2},
    {"zona": "A27", "rentabilidade": 1700, "impacto": 1},
    {"zona": "A28", "rentabilidade": 1700, "impacto": 3},
    {"zona": "A29", "rentabilidade": 1500, "impacto": 2},
    {"zona": "A30", "rentabilidade": 1700, "impacto": 3}
]

### Funções

In [3]:
# função de avaliação da solução
def f(solucao):
    return sum(zona["rentabilidade"] for zona in solucao)


# função para obter as zonas adjacentes
def zonas_adjacentes(zona):
    adj = {
        "A1": ["A2", "A7"], 
        "A2": ["A1", "A8", "A3", "A4"], 
        "A3": ["A2", "A4", "A5"],
        "A4": ["A2", "A3", "A5", "A9", "A10"], 
        "A5": ["A3", "A4", "A11", "A6"],
        "A6": ["A5", "A12"],
        "A7": ["A1", "A8", "A13"],
        "A8": ["A7", "A2", "A9", "A13", "A14"],
        "A9": ["A8", "A4", "A10", "A14", "A15"],
        "A10": ["A9", "A4", "A11", "A15"],
        "A11": ["A10", "A5", "A12", "A16"],
        "A12": ["A17", "A11", "A6"],
        "A13": ["A18", "A14", "A8", "A7"],
        "A14": ["A13", "A19", "A15", "A8", "A9"],
        "A15": ["A14", "A21", "A16", "A9", "A10"],
        "A16": ["A15", "A17", "A11", "A23"], 
        "A17": ["A16", "A12", "A25"], 
        "A18": ["A13", "A19", "A20", "A26"],
        "A19": ["A14", "A18", "A20", "A21"], 
        "A20": ["A18", "A19", "A22", "A27"],
        "A21": ["A19", "A15", "A23", "A22"],
        "A22": ["A20", "A21", "A24", "A28"],
        "A23": ["A21", "A16", "A25", "A24"],
        "A24": ["A22", "A23", "A25", "A29"],
        "A25": ["A17", "A23", "A24", "A30"],
        "A26": ["A18", "A27"],
        "A27": ["A26", "A20", "A28"],
        "A28": ["A27", "A22", "A29"],
        "A29": ["A28", "A24", "A30"],
        "A30": ["A29", "A25"]
    }
    return adj.get(zona["zona"], [])


# função para gerar um vizinho
def gerar_vizinho(solucao):
    while True:
        zona_a_substituir = random.choice(solucao)
        solucao_resto = [zona for zona in solucao if zona != zona_a_substituir]
        zonas_disponiveis = [
            zona for zona in zonas 
            if zona not in solucao_resto and
               all(zona["zona"] not in zonas_adjacentes(z) for z in solucao_resto)
        ]
        if not zonas_disponiveis:
            continue
        nova_zona = random.choice(zonas_disponiveis)
        nova_solucao = solucao_resto + [nova_zona]
        
        impacto_total = sum(zona["impacto"] for zona in nova_solucao)

        check_areas1 = sorted([zona["zona"] for zona in solucao]) 
        check_areas2 = sorted([zona["zona"] for zona in nova_solucao])

        if (impacto_total <= 8) and (check_areas1 != check_areas2): # 2ª condição serve para garantir que a nova solução é diferente da que foi passada como parâmetro
            return nova_solucao


# função do simulated annealing
def simulated_annealing(solucao_inicial):
    t0 = 0.2 * f(solucao_inicial)
    T = [t0 * (0.5 ** k) for k in range(5)]
    mk = 5

    solucao_atual = solucao_inicial
    melhor_solucao = solucao_inicial
    melhor_rentabilidade = f(solucao_inicial)

    all_sols = pd.DataFrame(columns=['iteracao', 'temperatura', 'localizacao', 'rentabilidade', 
                                     'impacto ambiental'])

    for k in range(5):
        for i in range(mk):
            solucao_vizinha = gerar_vizinho(solucao_atual)
            rentabilidade_vizinho = f(solucao_vizinha)
            delta = f(solucao_atual) - rentabilidade_vizinho # por querermos maximizar a rentabilidade esperada

            probabilidade_aceitacao = math.exp(-delta / T[k]) if delta > 0 else 1
            aceitar = delta < 0 or probabilidade_aceitacao > random.random()

            if aceitar:
                solucao_atual = solucao_vizinha
                if rentabilidade_vizinho > melhor_rentabilidade:
                    melhor_solucao = solucao_vizinha
                    melhor_rentabilidade = rentabilidade_vizinho

            # guardar as iterações do algoritmo 
            new_row = pd.DataFrame([{
                'iteracao': k * mk + i,
                'temperatura': T[k],
                'localizacao': [zona["zona"] for zona in solucao_vizinha],
                'rentabilidade': f(solucao_vizinha),
                'impacto ambiental': sum(zona["impacto"] for zona in solucao_vizinha)
            }])
            all_sols = pd.concat([all_sols, new_row], ignore_index=True)

    return melhor_solucao, melhor_rentabilidade, all_sols

### Execução

In [4]:
##### como em python os índices começam a partir de 0, para obter, por exemplo, a zona A1 faz-se zonas[0]; A2 <- zonas[1]; A3 <- zonas[2]; ... ; A30 <- zonas[29] (isto é aplicável a todas as funções desenvolvidas) #####
sol_inicial = [zonas[0], zonas[29], zonas[7], zonas[19]] # esta variável pode ser modificada com outra zona qualquer para gerar outros valores no total da rentabilidade esperada e outro vizinho admissível (correr várias x a mesma função com o mesmo input também fará com que se gerem diferentes resultados pela componente de aleatoriedade implementada)

# função para calcular o total da rentabilidade esperada 
print("O valor da solução é", f(sol_inicial), '€\n')

# imprimir uma solução vizinha admissível
print('Solução vizinha admissível à apresentada é:')
nova_solucao = gerar_vizinho(sol_inicial)
for zona in nova_solucao:
    print(json.dumps(zona, indent=4))


##### zonas adjacentes 
num_zona = 3 # variável pode ser alterada para ver as zonas adjacentes de outras zonas (neste caso, ao contrario dos exemplos acima 1 corresponde à zona A1; 2 <- A2; 3 <- A3; ... ; 30 <- A30)
print(f'\nAs zonas adjacentes de A{num_zona} são {zonas_adjacentes(zonas[num_zona-1])}')

O valor da solução é 12800 €

Solução vizinha admissível à apresentada é:
{
    "zona": "A30",
    "rentabilidade": 1700,
    "impacto": 3
}
{
    "zona": "A8",
    "rentabilidade": 4300,
    "impacto": 2
}
{
    "zona": "A20",
    "rentabilidade": 4200,
    "impacto": 1
}
{
    "zona": "A15",
    "rentabilidade": 1900,
    "impacto": 1
}

As zonas adjacentes de A3 são ['A2', 'A4', 'A5']


In [5]:
melhor_sol, melhor_val, all_sols = simulated_annealing([zonas[0], zonas[29], zonas[7], zonas[19]])

print('A melhor solução admissível obtida ao fim das mk iterações é:')
for zona in melhor_sol:
    print(json.dumps(zona, indent=4))

print(f"\nA maior rentabilidade esperada obtida ao fim de mk iterações é {melhor_val} €")

A melhor solução admissível obtida ao fim das mk iterações é:
{
    "zona": "A20",
    "rentabilidade": 4200,
    "impacto": 1
}
{
    "zona": "A24",
    "rentabilidade": 1800,
    "impacto": 1
}
{
    "zona": "A8",
    "rentabilidade": 4300,
    "impacto": 2
}
{
    "zona": "A12",
    "rentabilidade": 4700,
    "impacto": 4
}

A maior rentabilidade esperada obtida ao fim de mk iterações é 15000 €


In [6]:
# guardar o ficheiro txt com as iterações do algoritmo
all_sols.to_csv('output_sim_annealing.txt', sep='\t', index=False)